In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

> ** 몇 가지 추가로 필요한 모듈을 import 합니다.**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import explained_variance_score
from sklearn.metrics import confusion_matrix

> **import하는 모듈에 대한 간단한 설명입니다.**
> * **Pandas 판다스 : 마치 데이터베이스터럼 데이터를 읽어들이고 유지하고 관리**
> * **NumPy 넘파이 : 마치 계산기처럼 다양한 수치연산, 변환 기능 등 ** 
> * **Seaborn 시본 : 데이터 표시 기능**
> * **sklearn 싸이킷런 : 머신러닝 모델**

> **다음은 CSV(Comma-Separated Values, 콤마로 분류되는 값을 갖는) 파일의 예입니다. **
![](https://i.imgur.com/KQGT0La.png)

> **아래의 코드는 판다스를 이용하여 CSV 파일에 있는 데이터를 읽어들이고, 정보를 표시하고 있습니다.**

In [ ]:
pd.set_option('display.max_row', 1000)
pd.set_option('display.max_columns', 10)

In [ ]:
df = pd.read_csv('../input/kc_house_data.csv') #df = data frame
df.info()

> ** 실행결과를 보면 다음과 같은 내용을 알 수 있겠네요.**
* **컬럼(특징) 수가 모두 21개 (ID: 주택ID, Date: 거래일, Price: 주택가격, Bedrooms: 방 수, Bathrooms: 욕실 수, Sqft_Living: 주택 크기, Sqft_Lot: 차고지 면적, Floors: 층 수, Waterfront: 호수(바다)전망, View: 뷰, Condition: 상태, Grade: 평가점수, Sqft_Above: 지상면적, Sqft_Basement: 지하면적, Yr_Built: 건축년도, Yr_Renovated: 리모델링 년도, Zipcode: 우편번호, Lat: 위도, Long: 경도, Sqft_Living15: 2015년도 면적, Sqft_Lot15: 2015년도 차고지 면적**
* **거래정보(데이터) 수가 자그마치 21,613개**
* **대부분 숫자 데이터 : float64(5), int64(15), object(1)**
* **비어있는 데이터가 없음.** 

> **비어있는 데이터가 없으니 다행입니다. 이제 각 컬럼(특징)별로 유일한 값이 몇개나 있는지를 확인해보겠습니다.**

In [ ]:
for value in df:
    print('For {}, {} unique values present'.format(value, df[value].nunique()))

> **가령 grade는 12가지가 있네요. condition은 5가지,  호수(바다)전망은 있다/없다 2가지 등을 알 수 있습니다**

> **아래 코드는 21,613개 데이터 중 처음 10개만을 표시해봅니다.**

In [ ]:
df.head(10)

> **주택 가격에 id와 date는 아무 영향을 미치지 않을 것이라는 것을 예상할 수 있습니다. 따라서 이를 데이터 프레임에서 삭제할 수 있습니다.** 

In [ ]:
df = df.drop(['id','date'],axis=1)

In [ ]:
df.head(10)

> **id와 date가 사라졌습니다.**

> **위 실행 결과 첫 줄을 보면, 방이 3개, 화장실이 1개, 거실 크기가 1,180 sqft, 1층, 주차면적이 5,650 sqft 등인 집의 가격은 221,900 달러입니다. 2억 5천만원 정도가 되네요.**

> **자, 그럼 가격에 영향을 미치는 것이



In [ ]:
plt.figure(figsize=(10,6))
sns.plotting_context('notebook',font_scale=1.2)
g = sns.pairplot(df[['sqft_lot','sqft_above','price','sqft_living',
                     'bedrooms','grade','yr_built','yr_renovated']],
                 hue='bedrooms',size=2)
g.set(xticklabels=[])

> **위 그림에서 주택 가격(price)과 상관 관계가 비교적 뚜렷이 나타나는 것은 무엇일까요?**

> **참고로, 카테고리형 데이터가 섞여 있는 경우에는 hue 인수에 카테고리 이름을 지정하면 그 카테고리 값에 따라 색상을 다르게 하여 표시할 수 있습니다. 예를 들어, 키와 발 사이즈의 관계 그래프를 표시할 때 성별에 따라 색을 달리 표시하는 것이 예입니다.**

In [ ]:
sns.jointplot(x='sqft_living',y='price',data=df,kind='reg',size=4)

> **데이터도 비교적 양호하고 상관 관계도 보이니 이 데이터를 이용하여 바로 예측해 보도록 하겠습니다. 우선,  데이터를 학습 데이터(80%)와 테스트 데이터(20%)로 나누겠습니다.**

In [ ]:
from sklearn.model_selection import train_test_split # 함수
train_data,test_data = train_test_split(df,train_size = 0.8,random_state=3) # 3=seed

> **위의 실행결과를 보니 뭔가를 경고하는 것 같네요. 실행하는데 문제가 없습니다. 가끔 이런 경고성 문구가 귀찮을 때가 있습니다. 이 경우에는 아래와 같이 코드를 넣어주면 이후 경고 메시지가 표시되지 않습니다.**

In [ ]:
import warnings
warnings.filterwarnings('ignore')

> **그러면 아래와 같이 동일한 코드를 다시 한번 더 실행해 보겠습니다. 경고 문자열이 표시되나요?**

In [ ]:
from sklearn.model_selection import train_test_split
train_data,test_data = train_test_split(df,train_size = 0.8,random_state=3) #80%->학습용

> **싸이킷 런, sklearn(https://scikit-learn.org/stable/), 이건 무엇일까요? **
> * **scikit-learn은 Machine Learning in Python**
> * **간단하고 효과적인 데이터 마이닝, 데이터 분석 도구**
> * **누구나 사용할 수 있고 다양한 곳에 재사용 가능**
> * **NumPy, SciPy, 그리고 matplotlib를 이용하여 구현**
> * **오픈소스이면서 상업적으로도 이용가능(BSD 라이센스)**

> **학습 데이터, 테스트 데이터가 준비되었습니다. 이제 무엇을 할까요? 예측 시스템을 만들어 보겠습니다.**

In [ ]:
from sklearn import linear_model
gildong = linear_model.LinearRegression()

> **Regression은 Re(다시) + 가는 것(gression)을 의미하는 것으로 다시 반복되는 어떤 경향, 섭리, 법칙 등을 의미합니다. 우리 말로는 '회귀'라고 합니다.**

> **봄/여름/가을/겨울/봄/여름... 계절이 끊임없이 바뀌는 것도 회귀, 태어나서 자라고 청년기를 거치면서 늙고 죽는 것도 회귀, 젊으면 힘이 세고, 아주 어리거나 나이가 많이 들면 힘이 약하는 것도 회귀... **

> **이러한 회귀는 결국 '지식(knowledge)'이 됩니다. 이런 지식, 회귀를 알아내는 것이 바로 '학습'입니다****

> **위 코드는 이러한 회귀를 학습하는 녀석(객체) gildong이를 만들었습니다.**

# **<span style="font-size:1.5em;">Feature (sqft_living, 주택 면적)</span>**


> **저 많은 21개 컬럼(특징)들 중에서 주택 면적(sqft_living) 정보를 입력으로 주었을 때 그에 따른 가격 정보를 출력하도록 학습시켜 예측해보겠습니다.**

In [ ]:
#학습 데이터(80%)에서 주택 면적만 뽑아서 입력 데이터로
X_train = np.array(train_data['sqft_living'], dtype=pd.Series) 
print(X_train, '\n', X_train.shape, '\n')
X_train = X_train.reshape(-1,1) # 하나짜리를 여러개로 만들어라.
print(X_train, '\n', X_train.shape)

> **정답(주택가격)도 뽑아냅니다.**

> **데이터 프레임에 있는 컬럼 중에서 sqft_living 컬럼만을 뽑아서 넘파이 배열로 만듭니다. 그러면 [1510 1970 1440 ... 2140 1210 610] 형태로 바뀝니다. 이를 reshape 합니다. (-1, 1)은 하나짜리가 여러 개 있도록 만들어라 하는 의미입니다.  그러면 위와 같은 형태로 바뀝니다. **

In [ ]:
Y_train = np.array(train_data['price'], dtype=pd.Series) #가격만 뽑아냄.
print(Y_train, '\n', Y_train.shape)

> **gildong이에게 입력으로 거실 면적(X_train)과 정답으로 집값(Y_train)을 쌍으로 주어 학습(fit)시킵니다.** 

In [ ]:
gildong.fit(X_train,Y_train)

In [ ]:
import time
start_time = time.time()

gildong.fit(X_train,Y_train)

print("training Runtime: %0.3f초"%((time.time() - start_time)))

> **학습이 끝났습니다. 매우 짧은 시간에 완료되었네요. 이제, 나머지 20% 테스트 데이터에서 주택 면적(sqft_living)를 뽑아서 가격을 예측(predict)하라고 해보겠습니다.**

In [ ]:
X_test = np.array(test_data['sqft_living'], dtype=pd.Series).reshape(-1,1)
print(X_test)

In [ ]:
predicted = gildong.predict(X_test)
print(predicted, '\n', predicted.shape)

> **예측이 끝났네요. 길동이는 위와 같이 4,323개의 주택 가격을 예측하였습니다. 얼마나 정확하게 예측했을까요?**

> **이를 위해 실제 주택 가격(정답)을 뽑아냅니다. 그리고 얼마나 유사한지 비교해보겠습니다. 아래 코드는 실제 주택 가격을 뽑아내는 코드입니다.** 

In [ ]:
Y_test = np.array(test_data['price'], dtype=pd.Series)
print(Y_test, '\n', Y_test.shape)

> **다음은 실제 주택가격과 예측한 가격의 차이(error)를 제곱(square)한 후 평균값(mean)을 구하는 코드입니다**


> **위에서 뽑아 놓은 거실 면적 데이터 4,323개(X_test)를 주면서 집 가격을 예측해보라고 합니다.**

In [ ]:
from sklearn import metrics
msesm = format(np.sqrt(metrics.mean_squared_error(Y_test,predicted)),'.3f')
print(msesm)
rtrsm = format(gildong.score(X_train, Y_train),'.3f')
print(rtrsm)
rtesm = format(gildong.score(X_test, Y_test),'.3f')
print(rtesm)

In [ ]:
print ("Average Price for predicted: {:.3f}".format(predicted.mean()))
print ("Average Price for Test Data: {:.3f}".format(Y_test.mean()))

> **선형 모델 직선의 기울기와 상수의 값을 출력해보면 다음과 같습니다.**

In [ ]:
print('Intercept: {}'.format(gildong.intercept_))
print('Coefficient: {}'.format(gildong.coef_))

In [ ]:
evaluation = pd.DataFrame(
    {'Model': [],
    'Details':[],
    'MSE':[],
    'R-squared (train)':[],
    'Adj. R-sq (train)':[],
    'R-sq':[],
    'Adj. R-sq':[]})

In [ ]:
r = evaluation.shape[0]
evaluation.loc[r] = ['Simple Model', '-', msesm, rtrsm, '-', rtesm, '-']
print(evaluation)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(7, 6))
plt.scatter(X_test,Y_test,color='darkgreen',label="Data", alpha=.1)
plt.plot(X_test,gildong.predict(X_test),color="red",label="Predicted Regression Line")
plt.xlabel("Living Space (sqft)", fontsize=15)
plt.ylabel("Price ($)", fontsize=15)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.legend()

In [ ]:
import seaborn as sns

sns.set(style="white", font_scale=1)

features = ['price','bedrooms','bathrooms','sqft_living','sqft_lot','floors',
            'waterfront','view','condition','grade','sqft_above','sqft_basement',
            'yr_built','yr_renovated','zipcode','sqft_living15','sqft_lot15']

mask = np.zeros_like(df[features].corr(), dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
f, ax = plt.subplots(figsize=(16, 12))
plt.title('Pearson Correlation Matrix',fontsize=25)

sns.heatmap(df[features].corr(),linewidths=0.25,vmax=1.0,square=True,cmap="BuGn_r",
            linecolor='w',annot=True,mask=mask,cbar_kws={"shrink": .75});

In [ ]:
plt.figure(figsize=(15,10))
columns =['price','bedrooms','bathrooms','sqft_living','floors','grade','yr_built','condition']
sns.heatmap(df[columns].corr(),annot=True)

In [ ]:
f, axes = plt.subplots(1, 2,figsize=(15,5))
sns.boxplot(x=train_data['bedrooms'],y=train_data['price'], ax=axes[0])
sns.boxplot(x=train_data['floors'],y=train_data['price'], ax=axes[1])
axes[0].set(xlabel='Bedrooms', ylabel='Price')
axes[1].yaxis.set_label_position("right")
axes[1].yaxis.tick_right()
axes[1].set(xlabel='Floors', ylabel='Price')

f, axe = plt.subplots(1, 1,figsize=(12.18,5))
sns.boxplot(x=train_data['bathrooms'],y=train_data['price'], ax=axe)
axe.set(xlabel='Bathrooms / Bedrooms', ylabel='Price');

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig=plt.figure(figsize=(19,12.5))
ax=fig.add_subplot(2,2,1, projection="3d")
ax.scatter(train_data['floors'],train_data['bedrooms'],train_data['bathrooms'],c="darkgreen",alpha=.5)
ax.set(xlabel='\nFloors',ylabel='\nBedrooms',zlabel='\nBathrooms / Bedrooms')
ax.set(ylim=[0,12])

ax=fig.add_subplot(2,2,2, projection="3d")
ax.scatter(train_data['floors'],train_data['bedrooms'],train_data['sqft_living'],c="darkgreen",alpha=.5)
ax.set(xlabel='\nFloors',ylabel='\nBedrooms',zlabel='\nsqft Living')
ax.set(ylim=[0,12])

ax=fig.add_subplot(2,2,3, projection="3d")
ax.scatter(train_data['sqft_living'],train_data['sqft_lot'],train_data['bathrooms'],c="darkgreen",alpha=.5)
ax.set(xlabel='\n sqft Living',ylabel='\nsqft Lot',zlabel='\nBathrooms / Bedrooms')
ax.set(ylim=[0,250000])

ax=fig.add_subplot(2,2,4, projection="3d")
ax.scatter(train_data['sqft_living'],train_data['sqft_lot'],train_data['bedrooms'],c="darkgreen",alpha=.5)
ax.set(xlabel='\n sqft Living',ylabel='\nsqft Lot',zlabel='Bedrooms')
ax.set(ylim=[0,250000]);

In [ ]:
features1 = ['bedrooms','bathrooms','sqft_living','sqft_lot','floors','zipcode']
complex_model_1 = linear_model.LinearRegression()
complex_model_1.fit(train_data[features1],train_data['price'])


In [ ]:
print('Intercept: {}'.format(complex_model_1.intercept_))
print('Coefficients: {}'.format(complex_model_1.coef_))

In [ ]:
def adjustedR2(r2,n,k):
    return r2-(k-1)/(n-k)*(1-r2)

predicted1 = complex_model_1.predict(test_data[features1])
msecm1 = format(np.sqrt(metrics.mean_squared_error(Y_test, predicted1)),'.3f')
rtrcm1 = format(complex_model_1.score(train_data[features1],train_data['price']),'.3f')
artrcm1 = format(adjustedR2(complex_model_1.score(train_data[features1],train_data['price']),train_data.shape[0],len(features1)),'.3f')
rtecm1 = format(complex_model_1.score(test_data[features1],test_data['price']),'.3f')
artecm1 = format(adjustedR2(complex_model_1.score(test_data[features1],test_data['price']),test_data.shape[0],len(features1)),'.3f')


In [ ]:
r = evaluation.shape[0]
evaluation.loc[r] = ['Complex Model-1','-',msecm1,rtrcm1,artrcm1,rtecm1,artecm1]
print(evaluation.sort_values(by = 'R-squared (test)', ascending=False))

In [ ]:
f, axes = plt.subplots(1, 2,figsize=(15,5))
sns.boxplot(x=train_data['waterfront'],y=train_data['price'], ax=axes[0])
sns.boxplot(x=train_data['view'],y=train_data['price'], ax=axes[1])
axes[0].set(xlabel='Waterfront', ylabel='Price')
axes[1].yaxis.set_label_position("right")
axes[1].yaxis.tick_right()
axes[1].set(xlabel='View', ylabel='Price')

f, axe = plt.subplots(1, 1,figsize=(12.18,5))
sns.boxplot(x=train_data['grade'],y=train_data['price'], ax=axe)
axe.set(xlabel='Grade', ylabel='Price');

In [ ]:
fig=plt.figure(figsize=(9.5,6.25))
ax=fig.add_subplot(1,1,1, projection="3d")
ax.scatter(train_data['view'],train_data['grade'],train_data['yr_built'],c="darkgreen",alpha=.5)
ax.set(xlabel='\nView',ylabel='\nGrade',zlabel='\nYear Built');


In [ ]:
features2 = ['bedrooms','bathrooms','sqft_living','sqft_lot','floors','waterfront','view',
             'grade','yr_built','zipcode']
complex_model_2 = linear_model.LinearRegression()
complex_model_2.fit(train_data[features2],train_data['price'])

print('Intercept: {}'.format(complex_model_2.intercept_))
print('Coefficients: {}'.format(complex_model_2.coef_))

In [ ]:
pred2 = complex_model_2.predict(test_data[features2])
msecm2 = format(np.sqrt(metrics.mean_squared_error(Y_test,pred2)),'.3f')
rtrcm2 = format(complex_model_2.score(train_data[features2],train_data['price']),'.3f')
artrcm2 = format(adjustedR2(complex_model_2.score(train_data[features2],train_data['price']),train_data.shape[0],len(features2)),'.3f')
rtecm2 = format(complex_model_2.score(test_data[features2],test_data['price']),'.3f')
artecm2 = format(adjustedR2(complex_model_2.score(test_data[features2],test_data['price']),test_data.shape[0],len(features2)),'.3f')

r = evaluation.shape[0]
evaluation.loc[r] = ['Complex Model-2','-',msecm2,rtrcm2,artrcm2,rtecm2,artecm2]
print(evaluation.sort_values(by = 'R-squared (test)', ascending=False))

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

polyfeat = PolynomialFeatures(degree=2)
X_trainpoly = polyfeat.fit_transform(train_data[features2])
X_testpoly = polyfeat.fit_transform(test_data[features2])
poly = linear_model.LinearRegression().fit(X_trainpoly, train_data['price'])

predp = poly.predict(X_testpoly)
msepoly1 = format(np.sqrt(metrics.mean_squared_error(test_data['price'],predp)),'.3f')
rtrpoly1 = format(poly.score(X_trainpoly,train_data['price']),'.3f')
rtepoly1 = format(poly.score(X_testpoly,test_data['price']),'.3f')

polyfeat = PolynomialFeatures(degree=3)
X_trainpoly = polyfeat.fit_transform(train_data[features2])
X_testpoly = polyfeat.fit_transform(test_data[features2])
poly = linear_model.LinearRegression().fit(X_trainpoly, train_data['price'])

predp = poly.predict(X_testpoly)
msepoly2 = format(np.sqrt(metrics.mean_squared_error(test_data['price'],predp)),'.3f')
rtrpoly2 = format(poly.score(X_trainpoly,train_data['price']),'.3f')
rtepoly2 = format(poly.score(X_testpoly,test_data['price']),'.3f')

r = evaluation.shape[0]
evaluation.loc[r] = ['Polynomial Regression','degree=2',msepoly1,rtrpoly1,'-',rtepoly1,'-']
evaluation.loc[r+1] = ['Polynomial Regression','degree=3',msepoly2,rtrpoly2,'-',rtepoly2,'-']
print(evaluation.sort_values(by = 'R-squared (test)', ascending=False))

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

knnreg = KNeighborsRegressor(n_neighbors=15)
knnreg.fit(train_data[features2],train_data['price'])
pred = knnreg.predict(test_data[features2])

mseknn1 = format(np.sqrt(metrics.mean_squared_error(Y_test,pred)),'.3f')
rtrknn1 = format(knnreg.score(train_data[features2],train_data['price']),'.3f')
artrknn1 = format(adjustedR2(knnreg.score(train_data[features2],train_data['price']),train_data.shape[0],len(features2)),'.3f')
rteknn1 = format(knnreg.score(test_data[features2],test_data['price']),'.3f')
arteknn1 = format(adjustedR2(knnreg.score(test_data[features2],test_data['price']),test_data.shape[0],len(features2)),'.3f')

knnreg = KNeighborsRegressor(n_neighbors=25)
knnreg.fit(train_data[features2],train_data['price'])
pred = knnreg.predict(test_data[features2])

mseknn2 = format(np.sqrt(metrics.mean_squared_error(Y_test,pred)),'.3f')
rtrknn2 = format(knnreg.score(train_data[features2],train_data['price']),'.3f')
artrknn2 = format(adjustedR2(knnreg.score(train_data[features2],train_data['price']),train_data.shape[0],len(features2)),'.3f')
rteknn2 = format(knnreg.score(test_data[features2],test_data['price']),'.3f')
arteknn2 = format(adjustedR2(knnreg.score(test_data[features2],test_data['price']),test_data.shape[0],len(features2)),'.3f')

knnreg = KNeighborsRegressor(n_neighbors=27)
knnreg.fit(train_data[features2],train_data['price'])
pred = knnreg.predict(test_data[features2])

mseknn3 = format(np.sqrt(metrics.mean_squared_error(Y_test,pred)),'.3f')
rtrknn3 = format(knnreg.score(train_data[features2],train_data['price']),'.3f')
artrknn3 = format(adjustedR2(knnreg.score(train_data[features2],train_data['price']),train_data.shape[0],len(features2)),'.3f')
rteknn3 = format(knnreg.score(test_data[features2],test_data['price']),'.3f')
arteknn3 = format(adjustedR2(knnreg.score(test_data[features2],test_data['price']),test_data.shape[0],len(features2)),'.3f')

r = evaluation.shape[0]
evaluation.loc[r] = ['KNN Regression','k=15',mseknn1,rtrknn1,artrknn1,rteknn1,arteknn1]
evaluation.loc[r+1] = ['KNN Regression','k=25',mseknn2,rtrknn2,artrknn2,rteknn2,arteknn2]
evaluation.loc[r+2] = ['KNN Regression','k=27',mseknn3,rtrknn3,artrknn3,rteknn3,arteknn3]
print(evaluation.sort_values(by = 'R-squared (test)', ascending=False))

> **이로서 학습을 수행한 후 예측하도록 해 봤습니다.**